# Near real-time sales forecasting leveraging Synapse Link for Azure Cosmos DB

## Key Information about this notebook

* This notebook is part of the Azure Synapse Link for Azure Cosmos DB analitycal sample notebooks. For more information, click [here](../../../README.md). 

* It was build for Azure Cosmos DB SQL API but you can, by yourself, customize it for Azure Cosmos DB API for MongoDB. Please read about the analytical store inference schema differences between these 2 APIs [here](https://docs.microsoft.com/azure/cosmos-db/analytical-store-introduction#analytical-schema). 

* This is a Synapse Notebook and it was created to run in Synapse Analytics workspaces. Please make sure that you followed the pre-reqs of the [README](/README.md) file. After that, please execute the steps below in the same order that they are presented here. 

* From now on, all operations are case sentitive. Please be careful with everything you need to type.

## Predictive Analytics

Predictive analytics can help us to study and discover the factors that determine the number of sales that a retail store will have in the future.

This notebook scenario is [Microsoft Surface](https://www.microsoft.com/en-us/surface) sales forecasting, with artificially created data. The business challenge is a **distributor that wants to predict how many units are necessary in the local warehouse to supply the stores in the area.**

We will use Quantitative Models to forecast future data as a function of past data. They are appropriate to use when past numerical data is available and when it is reasonable to assume that some of the patterns in the data are expected to continue into the future. These methods are usually applied to short or intermediate range decisions. For more information, click [here](https://en.wikipedia.org/wiki/Forecasting).


<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/store.PNG" alt="Surface Device" width="75%"/>

## Environment Creation

The required steps are listed below. Please check the availabilty and try to colocate all these services in the same Azure region. Also, if you already have any of these services, created in another sample notebook, it is not required to create new ones.

1. [README](/README.md) file pre-reqs.
1. Load the data using the [Batch Ingestion notebook](1CosmoDBSynapseSparkBatchIngestion.ipynb).
1. Create an Azure Machine Learning workspace using [this](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace) tutorial.
1. Create an Azure Automated Machine Learning experiment using [this](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-automated-ml-for-ml-models) tutorial. You don't need to run it, neither to create a compute cluster.
1. Go to your **Synapse Spark Pool**, click on the **Packages Settings** tab, upload the [req.txt](req.txt) file located in this same folder, and click **force to get new settings on the spark pool** option. 

## Leverage power of Spark SQL to join & aggregate operational data across Cosmos DB containers

In [ ]:
%%sql
create database if not exists RetailSalesDemoDB

In [ ]:
%%sql
create table if not exists RetailSalesDemoDB.RetailSales using cosmos.olap options (
    spark.synapse.linkedService 'RetailSalesDemoDB',
    spark.cosmos.preferredRegions 'West US 2',
    spark.cosmos.container 'RetailSales'
)

In [ ]:
%%sql
create table if not exists RetailSalesDemoDB.StoreDemographics using cosmos.olap options (
    spark.synapse.linkedService 'RetailSalesDemoDB',
    spark.cosmos.preferredRegions 'West US 2',
    spark.cosmos.container 'StoreDemoGraphics'
)

In [ ]:
%%sql
create table if not exists RetailSalesDemoDB.Product using cosmos.olap options (
    spark.synapse.linkedService 'RetailSalesDemoDB',
    spark.cosmos.preferredRegions 'West US 2',
    spark.cosmos.container 'Products'
)

In [ ]:
data = spark.sql("select a.storeId \
                       , b.productCode \
                       , b.wholeSaleCost \
                       , b.basePrice \
                       , c.ratioAge60 \
                       , c.collegeRatio \
                       , c.income \
                       , c.highIncome150Ratio \
                       , c.largeHH \
                       , c.minoritiesRatio \
                       , c.more1FullTimeEmployeeRatio \
                       , c.distanceNearestWarehouse \
                       , c.salesNearestWarehousesRatio \
                       , c.avgDistanceNearest5Supermarkets \
                       , c.salesNearest5StoresRatio \
                       , a.quantity \
                       , a.logQuantity \
                       , a.advertising \
                       , a.price \
                       , a.weekStarting \
                 from RetailSalesDemoDB.retailsales a \
                 left join RetailSalesDemoDB.product b \
                 on a.productcode = b.productcode \
                 left join RetailSalesDemoDB.storedemographics c \
                 on a.storeId = c.storeId \
                 order by a.weekStarting, a.storeId, b.productCode")

display(data)

## Leverage power of Azure Machine Learning's AutoML to build a Forecasting Model

### Setup 
Let's start with the AML Environment setup. Please replace all "your-AML-%" placeholders with your own information, that you can get from your AML workspace in the the Azure Portal.


In [12]:
import azureml.core
import pandas as pd
import numpy as np
import logging
from azureml.core.workspace import Workspace
from azureml.core import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
import os
subscription_id = os.getenv("SUBSCRIPTION_ID", default="your-AML-subscription-id")
resource_group = os.getenv("RESOURCE_GROUP", default="your-AML-resource-group")
workspace_name = os.getenv("WORKSPACE_NAME", default="your-AML-workspace-name")
workspace_region = os.getenv("WORKSPACE_REGION", default="your-AML-region")

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
ws.write_config()
    
experiment_name = 'automl-surfaceforecasting'
experiment = Experiment(ws, experiment_name)
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Subscription ID   220fc532-6091-423c-8ba0-66c2397d591b
Workspace         rosouz-ml                           
SKU               Enterprise                          
Resource Group    rosouz-analytics                    
Location          westus2                             
Run History Name  automl-surfaceforecasting

### Data Preparation - Feature engineering, Splitting train & test datasets


In [50]:
# Initial variables
time_column_name = 'weekStarting'
grain_column_names = ['storeId', 'productCode']
target_column_name = 'quantity'
use_stores = [2, 5, 8,71,102]
n_test_periods = 20


#DataFrame
df = data.toPandas()
df[time_column_name] = pd.to_datetime(df[time_column_name])
df['storeId'] = pd.to_numeric(df['storeId'])
df['quantity'] = pd.to_numeric(df['quantity'])
df['advertising'] = pd.to_numeric(df['advertising'])
df['price'] = df['price'].astype(float)
df['basePrice'] = df['basePrice'].astype(float)
df['ratioAge60'] = df['ratioAge60'].astype(float)
df['collegeRatio'] = df['collegeRatio'].astype(float)
df['highIncome150Ratio'] = df['highIncome150Ratio'].astype(float)
df['income'] = df['income'].astype(float)
df['largeHH'] = df['largeHH'].astype(float)
df['minoritiesRatio'] = df['minoritiesRatio'].astype(float)
df['logQuantity'] = df['logQuantity'].astype(float)
df['more1FullTimeEmployeeRatio'] = df['more1FullTimeEmployeeRatio'].astype(float)
df['distanceNearestWarehouse'] = df['distanceNearestWarehouse'].astype(float)
df['salesNearestWarehousesRatio'] = df['salesNearestWarehousesRatio'].astype(float)
df['avgDistanceNearest5Supermarkets'] = df['avgDistanceNearest5Supermarkets'].astype(float)
df['salesNearest5StoresRatio'] = df['salesNearest5StoresRatio'].astype(float)


# Time Series
data_subset = df[df.storeId.isin(use_stores)]
nseries = data_subset.groupby(grain_column_names).ngroups
print('Data subset contains {0} individual time-series.'.format(nseries))

# Group by date
def split_last_n_by_grain(df, n):
    """Group df by grain and split on last n rows for each group."""
    df_grouped = (df.sort_values(time_column_name) # Sort by ascending time
                  .groupby(grain_column_names, group_keys=False))
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail

# splitting
train, test = split_last_n_by_grain(data_subset, n_test_periods)
print(len(train),len(test))
train.to_csv (r'./SurfaceSales_train.csv', index = None, header=True)
test.to_csv (r'./SurfaceSales_test.csv', index = None, header=True)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./SurfaceSales_train.csv', './SurfaceSales_test.csv'], target_path = 'dataset/', overwrite = True,show_progress = True)

# loading the train dataset
from azureml.core.dataset import Dataset
train_dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('dataset/SurfaceSales_train.csv'))

Data subset contains 15 individual time-series.
1422 300
Uploading an estimated of 2 files
Uploading ./SurfaceSales_test.csv
Uploading ./SurfaceSales_train.csv
Uploaded ./SurfaceSales_test.csv, 1 files out of an estimated total of 2
Uploaded ./SurfaceSales_train.csv, 2 files out of an estimated total of 2
Uploaded 2 files

### Training the Models using AutoML Forecasting

Please notice that **compute_target** is commented, meaning that the model training will run locally in Synapse Spark.

In [51]:
# Parameters
time_series_settings = {
    'time_column_name': time_column_name,
    'grain_column_names': grain_column_names,
    'max_horizon': n_test_periods
}

# Config
automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl_ss_sales_errors.log',
                             primary_metric='normalized_mean_absolute_error',
                             experiment_timeout_hours=0.5,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             #compute_target=compute_target,
                             enable_early_stopping=True,
                             n_cross_validations=3,
                             verbosity=logging.INFO,
                             **time_series_settings)

# Running the training
remote_run = experiment.submit(automl_config, show_output=True)



Running on local machine
Parent Run ID: AutoML_5152d53c-3785-4995-ab9a-1b6bf42a4592

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The 

### Retrieving the Best Model and Forecasting

In [52]:
# Retrieving the best model
best_run, fitted_model = remote_run.get_output()
print(fitted_model.steps)
model_name = best_run.properties['model_name']
print(model_name)

# Forecasting based on test dataset
X_test = test
y_test = X_test.pop(target_column_name).values
X_test[time_column_name] = pd.to_datetime(X_test[time_column_name])
y_predictions, X_trans = fitted_model.forecast(X_test)

[('timeseriestransformer', TimeSeriesTransformer(featurization_config=None, logger=None,
           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)), ('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('14', Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0...    min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'))]))],
               flatten_transform=None,
               weights=[0.9333333333333333, 0.06666666666666667]))]
AutoML5152d53c330

### Plotting the Results

In [53]:
import pandas as pd
import numpy as np
from pandas.tseries.frequencies import to_offset


def align_outputs(y_predicted, X_trans, X_test, y_test, target_column_name,
                  predicted_column_name='predicted',
                  horizon_colname='horizon_origin'):
    """
    Demonstrates how to get the output aligned to the inputs
    using pandas indexes. Helps understand what happened if
    the output's shape differs from the input shape, or if
    the data got re-sorted by time and grain during forecasting.

    Typical causes of misalignment are:
    * we predicted some periods that were missing in actuals -> drop from eval
    * model was asked to predict past max_horizon -> increase max horizon
    * data at start of X_test was needed for lags -> provide previous periods
    """

    if (horizon_colname in X_trans):
        df_fcst = pd.DataFrame({predicted_column_name: y_predicted,
                                horizon_colname: X_trans[horizon_colname]})
    else:
        df_fcst = pd.DataFrame({predicted_column_name: y_predicted})

    # y and X outputs are aligned by forecast() function contract
    df_fcst.index = X_trans.index

    # align original X_test to y_test
    X_test_full = X_test.copy()
    X_test_full[target_column_name] = y_test

    # X_test_full's index does not include origin, so reset for merge
    df_fcst.reset_index(inplace=True)
    X_test_full = X_test_full.reset_index().drop(columns='index')
    together = df_fcst.merge(X_test_full, how='right')

    # drop rows where prediction or actuals are nan
    # happens because of missing actuals
    # or at edges of time due to lags/rolling windows
    clean = together[together[[target_column_name,
                               predicted_column_name]].notnull().all(axis=1)]
    return(clean)


df_all = align_outputs(y_predictions, X_trans, X_test, y_test, target_column_name)

from azureml.automl.runtime.shared.score import scoring
from matplotlib import pyplot as plt
from automl.client.core.common import constants

# use automl scoring module
scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all['predicted'],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET))

print("[Test data scores]\n")
for key, value in scores.items():    
    print('{}:   {:.3f}'.format(key, value))
    
# Plot outputs
#%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='b')
test_test = plt.scatter(df_all[target_column_name], df_all[target_column_name], color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()

[Test data scores]

root_mean_squared_error:   9.510
normalized_root_mean_squared_error:   0.006
normalized_root_mean_squared_log_error:   0.002
r2_score:   0.999
root_mean_squared_log_error:   0.012
explained_variance:   0.999
median_absolute_error:   0.301
mean_absolute_error:   2.822
mean_absolute_percentage_error:   0.671
normalized_mean_absolute_error:   0.002
spearman_correlation:   1.000
normalized_median_absolute_error:   0.000


## Closing

At this point you should have a chart like this one in the image below, created wit AutoML and MatplotLib. The results are that good because of the **logQuantity** column, a  data Leakage calculated from que **quantity** column. You can try to run the same experiment without it.

<img src="https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/prediction.PNG" alt="Chart" width="75%"/>

## Next Steps

1. [Deploy](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where) your Model.
1. [Collect and Evaluate](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-enable-data-collection) Model Data.
1. [Create](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-your-first-pipeline) a ML Pipeline.
1. [Create](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-event-grid) Events Driven CI/CD Workflows.

